In [1]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import numpy as np
from PIL import Image
import io
import cv2 as cv
from __future__ import print_function
from google.colab import drive
drive.mount("/content/drive")
def VideoCapture():
  js = Javascript('''
    async function create(){
      div = document.createElement('div');
      document.body.appendChild(div);

      video = document.createElement('video');
      video.setAttribute('playsinline', '');

      div.appendChild(video);

      stream = await navigator.mediaDevices.getUserMedia({video: {facingMode: "environment"}});
      video.srcObject = stream;

      await video.play();

      canvas =  document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);

      div_out = document.createElement('div');
      document.body.appendChild(div_out);
      img = document.createElement('img');
      div_out.appendChild(img);
    }

    async function capture(){
        return await new Promise(function(resolve, reject){
            pendingResolve = resolve;
            canvas.getContext('2d').drawImage(video, 0, 0);
            result = canvas.toDataURL('image/jpeg', 0.8);
            pendingResolve(result);
        })
    }

    function showimg(imgb64){
        img.src = "data:image/jpg;base64," + imgb64;
    }

  ''')
  display(js)




Mounted at /content/drive


In [2]:
def byte2image(byte):
  jpeg = b64decode(byte.split(',')[1])
  im = Image.open(io.BytesIO(jpeg))
  return np.array(im)

def image2byte(image):
  image = Image.fromarray(image)
  buffer = io.BytesIO()
  image.save(buffer, 'jpeg')
  buffer.seek(0)
  x = b64encode(buffer.read()).decode('utf-8')
  return x

Function used to convert byte and picture to each other

In [3]:
## Code to finish ...
def detect(img, cascade):
    rects = cascade.detectMultiScale(img, scaleFactor=1.3, minNeighbors=4, minSize=(30, 30),
                                     flags=cv.CASCADE_SCALE_IMAGE)
    if len(rects) == 0:
        return []
    rects[:,2:] += rects[:,:2]
    return rects


Fonction to detect the eyes and face

In [4]:
face_cascade = cv.CascadeClassifier("/content/drive/MyDrive/mam4_CV/haarcascade_frontalface_alt.xml")
eye_cascade = cv.CascadeClassifier('/content/drive/MyDrive/mam4_CV/haarcascade_eye.xml')

Load cascades using v2.CascadeClassifier

In [8]:
VideoCapture()
eval_js('create()') #Turn on the camera

while True: 
  byte = eval_js('capture()') #screenshot
  im = byte2image(byte) 
  gray = cv.cvtColor(im, cv.COLOR_RGB2GRAY)  #Convert ordinary pictures into grayscale images for computer processing
  gray = cv.equalizeHist(gray) #Count the values of these pixels in the image to get a unified overall gray concept
  rects = detect(gray,face_cascade) #Detect the position of the face
  print(rects)
  for (x, y, w, h) in rects:
    cv.rectangle(im, (x, y), (w, h), (0, 0, 255), 3) #Draw face
    face = gray[y:h,x:w] #Get a grayscale image of the face
    vis_face = im[y:h,x:w] #Get a original image of the face
    eyes = detect(face, eye_cascade) #Detect the position of the eyes in the grayscale image of the face
    for (a, b, c, d) in eyes:
      cv.rectangle(vis_face, (a, b), (c, d), (0, 255, 0), 3 ) #Draw the position of the eyes in the original image of the face
     
  
  eval_js('showimg("{}")'.format(image2byte(im)))

<IPython.core.display.Javascript object>

[]
[]
[]
[[246  78 419 251]]
[[247  85 410 248]]
[[237  82 400 245]]
[[211  71 384 244]]
[[150  75 313 238]]
[[ 93  73 281 261]]
[[ 77  93 260 276]]
[[418  68 612 262]]
[[417  88 590 261]]


KeyboardInterrupt: ignored